In [1]:
#8.3 Project Presentations for WASHSTL201809DATA3 by Group H - Cody Braun, Austen Manser and Bill Nash
#grabSongFeatures_w_Fuzzy_Match.ipynb

#import necessary packages
import spotipy
from spotipy import util
from pprint import pprint
import random
import string
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import json
import os
import csv

In [3]:
#generate token for spotify API
username = 'ernyjyqoecerasugt8dpmhbmo'
scope = 'playlist-modify-public'
my_client_id = '83c842b2f4d447b6b2a19df91847487d'
my_client_secret = '8bd6101237114d9eb851e2d3183affce'

spotify_token = util.prompt_for_user_token(username, 
                                           scope, 
                                           client_id=my_client_id, 
                                           client_secret=my_client_secret,
                                           redirect_uri='http://example.com/callback/')

sp = spotipy.Spotify(auth=spotify_token)

In [4]:
#import Billboard CSV

ChartsDF = pd.read_csv("../Resources/Class_Hot100BBCharts_time.csv")
#ChartsDF.head()

In [5]:
#Only keep song Title, Artist, and peak chart position

ChartsDF_small = ChartsDF[["Title","Artist","Peak_Position"]]
ChartsDF_small["Song, Artist"] = ChartsDF_small["Title"]+", "+ChartsDF_small["Artist"]
#ChartsDF_small.head()

/Users/william_nashs_mac/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
#Group DF by Song, Artist
GroupedChartsDF = ChartsDF_small.groupby(['Song, Artist']).min()
#GroupedChartsDF.head()

In [7]:
#Create column to store results of fuzzy match

dfcolumnsList = ["Song ID","Song Popularity","Acousticness","Danceability","Energy","Liveness","Speechiness","Valence","Tempo",
                "Instrumentalness","Loudness"]
GroupedChartsDF = GroupedChartsDF.reindex(columns=[*GroupedChartsDF.columns.tolist(), *dfcolumnsList])

#GroupedChartsDF

In [8]:
#Loop through songs and find best match from Spotify API
song_ids = []
songPopularity = []
acousticness = []
danceability = []
energy = []
liveness = []
speechiness = []
valence = []
tempo = []
instrumentalness = []
loudness =[]

attempts = 3000
songcounter = 0

for index, row in GroupedChartsDF.iterrows():
    
    #index contains: song tile and artist "Africa, Weezer"
    
    songcounter += 1
    print(f"\n Fetching for: {songcounter}:: {index}\n")
    
    if attempts == 3000:
        spotify_token = util.prompt_for_user_token(username, 
                                           scope, 
                                           client_id=my_client_id, 
                                           client_secret=my_client_secret,
                                           redirect_uri='http://example.com/callback/')

        sp = spotipy.Spotify(auth=spotify_token)
        
        attempts = 0
        
        print("token reset")
    
    attempts += 1
    
    test_song = index

    results = sp.search(q=test_song, type='track', limit=10)

    search_match_score = []
    
    if results is None:
        
        song_ids.append("")
        songPopularity.append("")
        danceability.append("")
        energy.append("")
        loudness.append("")
        speechiness.append("")
        acousticness.append("")
        instrumentalness.append("")
        liveness.append("")
        valence.append("")
        tempo.append("") 
        
    else:
        
        if results["tracks"]["total"] > 0:
            
            for x in range(len(results["tracks"]["items"])):
                song = results["tracks"]["items"][x]["name"]
                artist = results["tracks"]["items"][x]["artists"][0]["name"]
                song_id = results["tracks"]["items"][x]["id"]
                song_popularity = results["tracks"]["items"][x]["popularity"]            
                search_match_score.append(fuzz.ratio(test_song,song+","+artist))

            max_loc = 0
            max_val = 0
            
            for y in range(len(results["tracks"]["items"])):
                
                if max_val < search_match_score[y]:
                    max_loc = y
                    max_val = search_match_score[y]
        
            if max_val >= 75:
            
                #print(f"SongID after max_val: {song_id}\n")
                song_ids.append(song_id)
                songPopularity.append(song_popularity)
              
                features = sp.audio_features(song_id)
                
                xstuff = features
            
                if str(xstuff) == "[None]":    
            
                    danceability.append("")
                    energy.append("")
                    loudness.append("")
                    speechiness.append("")
                    acousticness.append("")
                    instrumentalness.append("")
                    liveness.append("")
                    valence.append("")
                    tempo.append("")
            
                else:  
                    for feature in features:
                        #print(f"songID: {song_id}\n")
                        #print(json.dumps(feature, indent=4))
                    
                        danceability.append(feature["danceability"])
                        energy.append(feature["energy"])
                        loudness.append(feature["loudness"])
                        speechiness.append(feature["speechiness"])
                        acousticness.append(feature["acousticness"])
                        instrumentalness.append(feature["instrumentalness"])
                        liveness.append(feature["liveness"])
                        valence.append(feature["valence"])
                        tempo.append(feature["tempo"]) 
            else:
                song_ids.append("")
                songPopularity.append("")
                danceability.append("")
                energy.append("")
                loudness.append("")
                speechiness.append("")
                acousticness.append("")
                instrumentalness.append("")
                liveness.append("")
                valence.append("")
                tempo.append("") 
        
        else:
            song_ids.append("")
            songPopularity.append("")
            danceability.append("")
            energy.append("")
            loudness.append("")
            speechiness.append("")
            acousticness.append("")
            instrumentalness.append("")
            liveness.append("")
            valence.append("")
            tempo.append("") 
        
        
GroupedChartsDF["Song ID"] = song_ids
GroupedChartsDF["Song Popularity"] = songPopularity
GroupedChartsDF["Acousticness"] = acousticness
GroupedChartsDF["Danceability"] = danceability
GroupedChartsDF["Energy"] = energy
GroupedChartsDF["Liveness"] = liveness
GroupedChartsDF["Speechiness"] = speechiness
GroupedChartsDF["Valence"] = valence
GroupedChartsDF["Tempo"] = tempo
GroupedChartsDF["Instrumentalness"] = instrumentalness
GroupedChartsDF["Loudness"] = loudness

GroupedChartsDF.to_csv('../Resources/Features_Class_Hot100BBCharts_time.csv', index=False)


print(f"That's all... done\n")
#GroupedChartsDF


 Fetching for: 1:: Africa, Weezer

token reset

 Fetching for: 2:: Always Remember Us This Way, Lady Gaga


 Fetching for: 3:: Astronauts, Future & Juice WRLD


 Fetching for: 4:: Back To You, Selena Gomez


 Fetching for: 5:: Backin' It Up, Pardison Fontaine Featuring Cardi B


 Fetching for: 6:: Beautiful, Bazzi Featuring Camila Cabello


 Fetching for: 7:: Best Part, Daniel Caesar Featuring H.E.R.


 Fetching for: 8:: Best Shot, Jimmie Allen


 Fetching for: 9:: Better Now, Post Malone


 Fetching for: 10:: Better, Khalid


 Fetching for: 11:: Big Bank, YG Featuring 2 Chainz, Big Sean & Nicki Minaj


 Fetching for: 12:: Biggest Alley Oop, Quavo


 Fetching for: 13:: Blue Tacoma, Russell Dickerson


 Fetching for: 14:: Boo'd Up, Ella Mai


 Fetching for: 15:: Break Up In The End, Cole Swindell


 Fetching for: 16:: Breathin, Ariana Grande


 Fetching for: 17:: Broken, lovelytheband


 Fetching for: 18:: Chanel (Go Get It), Young Thug Featuring Gunna & Lil Baby


 Fetching for: 19:: 